# Graphical Models for Textual Data
This shows how graphical models can be used to infer relationships between textual data.

In [48]:
from collections import Counter

import pandas as pd
from regain.utils import flatten
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

from regain.data import base; import imp; imp.reload(base)
train, test = base.load_webkb()

1 - create data manually (no filtering on terms)

In [103]:
# compute the dictionary of the words (only strings)
words = np.unique(flatten([words.split(' ') for words in train.words.tolist() if isinstance(words, str)]))

# for each document, create the TermFrequency
tf_docs = [dict(Counter(v.split(' '))) for k, v in train.itertuples() if isinstance(v, str)]
assert len(tf_docs) == len(train)

# build data
X = pd.DataFrame(tf_docs, index=[row.Index for row in train.itertuples()
      if isinstance(row.words, str)]).fillna(0)
y = X.index

2 - create data with some filters based on sklearn-`CountVectorizer`

In [104]:
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=1., min_df=0.) #, stop_words='english')
tf = tf_vectorizer.fit_transform(train.values.flatten())

df_tf = pd.DataFrame(tf.todense(), index=train.index, columns=tf_vectorizer.get_feature_names())

In [105]:
np.allclose(X, df_tf)

True

In [ ]:
# # NMF is able to use tf-idf
# tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
# tfidf = tfidf_vectorizer.fit_transform(documents)
# tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# # Run NMF
# nmf_model = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
# nmf_W = nmf_model.transform(tfidf)
# nmf_H = nmf_model.components_

# print("NMF Topics")
# display_topics(nmf_H, nmf_W, tfidf_feature_names, documents, no_top_words, no_top_documents)

In [86]:
n_topics = 50

In [87]:
# Run LDA
lda_model = LatentDirichletAllocation(n_components=n_topics, max_iter=5, learning_method='online',
                                      learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

In [96]:
from regain import utils_text; imp.reload(utils_text)

print("LDA Topics")
topics = utils_text.display_topics(lda_H, lda_W, tf_vectorizer.get_feature_names(),
                                   train.values.flatten(), n_top_words=4, n_top_documents=1, print_docs=False)

df = pd.DataFrame(lda_W, index=train.index, columns=topics)

LDA Topics
Topic 0: network (0.123) comput (0.104) research (0.098) paper (0.086)
Topic 1: austin (342.521) texa (185.231) utexa (150.192) alberta (132.321)
Topic 2: comput (381.630) scienc (352.159) robot (336.811) depart (216.900)
Topic 3: assign (0.098) mark (0.076) email (0.069) jonathan (0.063)
Topic 4: robbin (8.227) suni (5.420) flame (3.735) zhang (2.413)
Topic 5: pradesh (3.576) andhra (3.092) pankaj (2.803) hail (2.232)
Topic 6: homepag (45.555) game (15.986) iowa (8.438) othello (7.967)
Topic 7: linear (55.510) nonlinear (35.098) optim (29.788) mangasarian (27.749)
Topic 8: logic (145.654) china (35.050) wesleyan (29.713) file (29.278)
Topic 9: alistair (16.705) markov (15.683) clair (7.828) randal (5.716)
Topic 10: frame (113.834) browser (33.161) nmsu (33.044) mexico (28.437)
Topic 11: washington (296.118) seattl (107.628) bershad (61.874) sieg (48.006)
Topic 12: dsp (32.210) xxx (15.071) clayton (3.662) stoll (2.317)
Topic 13: swanson (0.738) lab (0.117) homework (0.109) 

## LogEntropyModel (`regain`)

In [106]:
words_to_discard = []
for yy in np.unique(y):
    words_to_discard += list(X[words[X[y==yy].sum(axis=0) == 0]].columns)

words_to_keep = list(set(X.columns) - set(words_to_discard))

In [107]:
X_new = utils_text.logentropy_normalize(X)[words_to_keep]

/Users/federicot/src/fdtomasi/regain/regain/utils_text.py:30: RuntimeWarning: divide by zero encountered in log
  E = 1 + (P * np.log(P)).fillna(0).values.sum(


## LogEntropyModel (`gensim`)

In [112]:
from gensim.models import LogEntropyModel, LdaModel
from gensim.test.utils import common_texts
from gensim.corpora import Dictionary

corp = [w.split(' ') for w in train.words if isinstance(w, str)]
text = corp #common_texts
dct = Dictionary(text)  # fit dictionary

In [120]:
num_terms = 50 # or words.size
dct.filter_extremes(keep_n=num_terms)

corpus = [dct.doc2bow(row) for row in text]  #convert to BoW format
model = LogEntropyModel(corpus, normalize=True)  # fit model

# model = models.LdaModel(corpus, id2word=dct, num_topics=num_terms)

In [124]:
import gensim
X_new = gensim.matutils.corpus2dense(model[corpus], num_terms=num_terms).T
df_new = pd.DataFrame(X_new, columns=list(dct.values()), index=y)

# df[words_to_keep].T.sort_index().T
# X = df[words_to_keep].values

# X = df.values
# y = df.index

In [125]:
from regain.covariance import kernel_time_graphical_lasso_
from regain.model_selection import stability_optimization
from sklearn.model_selection import StratifiedShuffleSplit

mdl = kernel_time_graphical_lasso_.KernelTimeGraphicalLasso(
    verbose=0, kernel=np.ones((np.unique(y).size, np.unique(y).size)), psi='l1',
    alpha=0.45, max_iter=1000).fit(X_new, y)

In [131]:
imp.reload(stability_optimization);
socv = stability_optimization.GraphicalModelStabilitySelection(
    mdl, param_grid=dict(alpha=np.logspace(2, -2)),
    sampling_size=200,
).fit(X_new, y)

In [132]:
n_times = np.unique(y).size
n_dim = X_new.shape[1]

In [133]:
idx = np.triu_indices(n_dim, 1)
dof = idx[0].size * n_times

In [134]:
mdl = socv.best_estimator_

In [135]:
print("Nonzero percentage: %.4f" % (np.sum([np.count_nonzero(P[idx]) for P in mdl.precision_]) / dof))

Nonzero percentage: 0.7133


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from kdge import plot_plotly
import plotly.offline as py
import plotly.graph_objs as go
from plotly import tools

py.init_notebook_mode()
import plotly.io as pio

In [ ]:
p = mdl.precision_[0]

In [ ]:
from regain.utils import retain_top_n

In [ ]:
k = -1
trace = []
graphs = []
# for i, p in enumerate(ltgl.precision_ - ltgl.latent_):
for i, p in enumerate(mdl.precision_):

    A = np.abs(p - np.diag(np.diag(p)))
    A = retain_top_n(A, top_n)
    G = nx.from_numpy_matrix(A * 0.00001)
    graphs.append(G)
    trace.append(pl.plot_circular(G, df.columns, 1.4, cmap='Blues',
                                  #color_nodes=plt.rcParams['axes.prop_cycle'].by_key()['color'][:5]
                                 ))

fig = tools.make_subplots(
    rows=1, cols=len(mdl.precision_), horizontal_spacing=.1, print_grid=False)

for j, tr in enumerate(trace):
    tr['data'][k]['xaxis'] = 'x' + str(j+1)
    tr['data'][k]['yaxis'] = 'y' + str(j+1)

for j, tr in enumerate(trace):
    for i, x in enumerate(tr['data']):
        col = j + 1
        x['legendgroup'] = 'group'+ str(j+1)
        x['showlegend'] = False
        fig.append_trace(x,1,col)
        


for j, tr in enumerate(trace):
    fig.layout.annotations += tuple([pl._set_ref(
        x, 'x'+ str(j+1), 'y'+ str(j+1)) for x in tr['layout']['annotations']])

    fig['layout']['xaxis'+str(j+1)].update(showgrid=False, zeroline=False, showticklabels=False)
    fig['layout']['yaxis'+str(j+1)].update(showgrid=False, zeroline=False, showticklabels=False)

py.init_notebook_mode()

fig['layout'].update(height=900, width=4000,hovermode='closest',
                     paper_bgcolor='rgba(0,0,0,0)',
                        plot_bgcolor='rgba(0,0,0,0)')
# fig.layout.annotations += tuple([dict(
#     text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
#     showarrow=False, xref="paper", yref="paper", x=0.005, y=-0.2)])
# fig['layout'].update(scene=dict(aspectmode="data"))
py.iplot(fig)
# py.offline.iplot(fig, filename='figure_factory_subplot', image='svg')

In [ ]:
pio.write_image(fig, "graphs.pdf")

## Correlated Topic Model

In [ ]:
import ctmmodel
ctm_model = ctmmodel.CtmModel(corpus, id2word=dct, num_topics=15)

all_words = []
for c in corpus:
    doc_words = []
    for cc in c:
        doc_words.extend([dct[cc[0]]] * cc[1])
    all_words.append(' '.join(doc_words))

In [ ]:
import pyctm
from pyctm import variational_bayes, inferencer, utils

In [ ]:
# parameter set 3
alpha_mu=0.
alpha_sigma=1
alpha_beta=0

ctm_inferencer = variational_bayes.VariationalBayes();
ctm_inferencer._initialize(
    all_words, list(dct.values()),
    number_of_topics=15,
    alpha_mu=alpha_mu, alpha_sigma=alpha_sigma,
    alpha_beta=alpha_beta);
# ctm_inferencer._initialize(train.words.tolist(), words, 20,
#                            alpha_mu, alpha_sigma, alpha_beta);

for iteration in range(50):
    ctm_inferencer.learning(-1)

logl, lamda, nu = ctm_inferencer.inference(all_words)
# logl, lamda, nu = ctm_inferencer.inference(train.words.tolist())

ll = utils.topic_beta(ctm_inferencer)

topic_words = pd.DataFrame(ll)

In [ ]:
topic_str_repr = []
for row in topic_words.iterrows():
    print(pd.DataFrame(row[1].sort_values(ascending=False)[:3]).T)
    topic_str_repr.append(' '.join(row[1].sort_values(ascending=False)[:3].index))

word_dct_values = list(dct.values())

dff = pd.DataFrame(ctm_model.beta, columns=word_dct_values)
dff = dff[sorted(dff.columns)]

In [ ]:
import numpy as np
from sklearn.covariance import GraphicalLassoCV

from regain import utils

In [ ]:
gl = GraphicalLassoCV().fit(lamda)

In [ ]:
p = gl.precision_

A = np.abs(p - np.diag(np.diag(p)))
A = retain_top_n(A, 20)
G = nx.from_numpy_matrix(A * 3)
fig = pl.plot_circular(G, topic_str_repr, 2, cmap='Blues')

fig['layout'].update(height=800, width=800,hovermode='closest',
                     paper_bgcolor='rgba(0,0,0,0)',
                        plot_bgcolor='rgba(0,0,0,0)')
# fig.layout.annotations += tuple([dict(
#     text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
#     showarrow=False, xref="paper", yref="paper", x=0.005, y=-0.2)])
# fig['layout'].update(scene=dict(aspectmode="data"))
py.iplot(fig)
# py.offline.iplot(fig, filename='figure_factory_subplot', image='svg')